In [ ]:
import numpy as np

def smape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    denom = np.abs(y_true) + np.abs(y_pred)
    denom = np.where(denom == 0, 1, denom)

    return np.mean(2.0 * np.abs(y_pred - y_true) / denom) * 100


In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_sub = pd.read_csv("sample_submission.csv")

if "Unnamed: 0" in train.columns:
    train = train.drop(columns=["Unnamed: 0"])

train["period_start_dt"] = pd.to_datetime(train["period_start_dt"])
test["period_start_dt"] = pd.to_datetime(test["period_start_dt"], dayfirst=True)

for df in (train, test):
    df["year"] = df["period_start_dt"].dt.year
    df["month"] = df["period_start_dt"].dt.month
    df["weekofyear"] = df["period_start_dt"].dt.isocalendar().week.astype(int)
    df["weekday"] = df["period_start_dt"].dt.weekday


In [ ]:
hist = train[~train["demand"].isna()].copy()

mean_ps = (
    hist.groupby(["product_rk", "store_location_rk"])["demand"]
    .mean()
    .reset_index()
    .rename(columns={"demand": "mean_ps"})
)

# product
mean_p = (
    hist.groupby("product_rk")["demand"]
    .mean()
    .reset_index()
    .rename(columns={"demand": "mean_p"})
)

# store
mean_s = (
    hist.groupby("store_location_rk")["demand"]
    .mean()
    .reset_index()
    .rename(columns={"demand": "mean_s"})
)

mean_ps_w = (
    hist.groupby(["product_rk", "store_location_rk", "weekofyear"])["demand"]
    .mean()
    .reset_index()
    .rename(columns={"demand": "mean_ps_w"})
)

mean_p_w = (
    hist.groupby(["product_rk", "weekofyear"])["demand"]
    .mean()
    .reset_index()
    .rename(columns={"demand": "mean_p_w"})
)

mean_s_w = (
    hist.groupby(["store_location_rk", "weekofyear"])["demand"]
    .mean()
    .reset_index()
    .rename(columns={"demand": "mean_s_w"})
)

global_mean = hist["demand"].mean()


In [ ]:
def add_group_means(df):
    out = df.copy()
    out = out.merge(mean_ps, on=["product_rk", "store_location_rk"], how="left")
    out = out.merge(mean_p, on="product_rk", how="left")
    out = out.merge(mean_s, on="store_location_rk", how="left")
    out = out.merge(mean_ps_w, on=["product_rk", "store_location_rk", "weekofyear"], how="left")
    out = out.merge(mean_p_w, on=["product_rk", "weekofyear"], how="left")
    out = out.merge(mean_s_w, on=["store_location_rk", "weekofyear"], how="left")

    for col in ["mean_ps", "mean_p", "mean_s", "mean_ps_w", "mean_p_w", "mean_s_w"]:
        out[col] = out[col].fillna(global_mean)

    return out

train_fe = add_group_means(train)
test_fe  = add_group_means(test)


In [ ]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

features = [
    "product_rk", "store_location_rk",
    "year", "month", "weekofyear", "weekday",
    "mean_ps", "mean_p", "mean_s",
    "mean_ps_w", "mean_p_w", "mean_s_w"
]

target = "demand"

df_train = train_fe[~train_fe["demand"].isna()].copy()

cutoff_date = pd.to_datetime("2019-09-01")
mask_train = df_train["period_start_dt"] < cutoff_date
mask_val   = df_train["period_start_dt"] >= cutoff_date

X_train = df_train.loc[mask_train, features]
y_train = df_train.loc[mask_train, target]
X_val   = df_train.loc[mask_val, features]
y_val   = df_train.loc[mask_val, target]

model = CatBoostRegressor(
    depth=8,
    learning_rate=0.05,
    l2_leaf_reg=3,
    iterations=1500,
    loss_function="RMSE",      
    eval_metric="MAE",         
    random_seed=42,
    verbose=200
)

model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    cat_features=[features.index("product_rk"), features.index("store_location_rk")]
)

pred_val = model.predict(X_val)
pred_val = np.clip(pred_val, 0, None)

mae = mean_absolute_error(y_val, pred_val)
print("Validation MAE:", mae)


0:	learn: 12.1682441	test: 9.1239469	best: 9.1239469 (0)	total: 68.9ms	remaining: 1m 43s
200:	learn: 4.1748930	test: 3.9211863	best: 3.8771706 (169)	total: 1.64s	remaining: 10.6s
400:	learn: 3.8628543	test: 4.0936758	best: 3.8771706 (169)	total: 3.07s	remaining: 8.42s
600:	learn: 3.6678852	test: 4.1232104	best: 3.8771706 (169)	total: 4.7s	remaining: 7.03s
800:	learn: 3.5257838	test: 4.0984920	best: 3.8771706 (169)	total: 6.92s	remaining: 6.04s
1000:	learn: 3.3912608	test: 4.0869446	best: 3.8771706 (169)	total: 8.67s	remaining: 4.32s
1200:	learn: 3.2665009	test: 4.0683617	best: 3.8771706 (169)	total: 10.5s	remaining: 2.61s
1400:	learn: 3.1545922	test: 4.0553383	best: 3.8771706 (169)	total: 12.6s	remaining: 889ms
1499:	learn: 3.1104111	test: 4.0569410	best: 3.8771706 (169)	total: 13.3s	remaining: 0us

bestTest = 3.877170613
bestIteration = 169

Shrink model to first 170 iterations.
Validation MAE: 3.870884051876643


In [ ]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

features = [
    "product_rk", "store_location_rk",
    "year", "month", "weekofyear", "weekday",
    "mean_ps", "mean_p", "mean_s",
    "mean_ps_w", "mean_p_w", "mean_s_w"
]

target = "demand"

df_train = train_fe[~train_fe["demand"].isna()].copy()

cutoff_date = pd.to_datetime("2019-09-01")
mask_train = df_train["period_start_dt"] < cutoff_date
mask_val   = df_train["period_start_dt"] >= cutoff_date

X_train = df_train.loc[mask_train, features]
y_train = df_train.loc[mask_train, target]
X_val   = df_train.loc[mask_val, features]
y_val   = df_train.loc[mask_val, target]

model = CatBoostRegressor(
    depth=8,
    learning_rate=0.05,
    l2_leaf_reg=3,
    iterations=1500,
    loss_function="RMSE",      
    eval_metric="MAE",         
    random_seed=42,
    verbose=200
)

model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    cat_features=[features.index("product_rk"), features.index("store_location_rk")]
)

pred_val = model.predict(X_val)
pred_val = np.clip(pred_val, 0, None)  


mae = mean_absolute_error(y_val, pred_val)
print("Validation MAE:", mae)

val_smape = smape(y_val, pred_val)
print("Validation SMAPE:", val_smape)


0:	learn: 12.1682441	test: 9.1239469	best: 9.1239469 (0)	total: 11.1ms	remaining: 16.6s
200:	learn: 4.1748930	test: 3.9211863	best: 3.8771706 (169)	total: 1.81s	remaining: 11.7s
400:	learn: 3.8628543	test: 4.0936758	best: 3.8771706 (169)	total: 3.62s	remaining: 9.93s
600:	learn: 3.6678852	test: 4.1232104	best: 3.8771706 (169)	total: 5.54s	remaining: 8.28s
800:	learn: 3.5257838	test: 4.0984920	best: 3.8771706 (169)	total: 7.19s	remaining: 6.28s
1000:	learn: 3.3912608	test: 4.0869446	best: 3.8771706 (169)	total: 8.84s	remaining: 4.41s
1200:	learn: 3.2665009	test: 4.0683617	best: 3.8771706 (169)	total: 10.5s	remaining: 2.6s
1400:	learn: 3.1545922	test: 4.0553383	best: 3.8771706 (169)	total: 12.2s	remaining: 860ms
1499:	learn: 3.1104111	test: 4.0569410	best: 3.8771706 (169)	total: 13s	remaining: 0us

bestTest = 3.877170613
bestIteration = 169

Shrink model to first 170 iterations.
Validation MAE: 3.870884051876643
Validation SMAPE: 90.64405063320461


In [ ]:
X_full = df_train[features]
y_full = df_train[target]

model.fit(
    X_full, y_full,
    cat_features=[features.index("product_rk"), features.index("store_location_rk")],
    verbose=200
)

X_test = test_fe[features]
test_pred = model.predict(X_test)
test_pred = np.clip(test_pred, 0, None)  


0:	learn: 11.5963704	total: 15.4ms	remaining: 23s
200:	learn: 4.1401978	total: 1.87s	remaining: 12.1s
400:	learn: 3.8312715	total: 3.39s	remaining: 9.28s
600:	learn: 3.6212149	total: 5.12s	remaining: 7.66s
800:	learn: 3.4768438	total: 6.95s	remaining: 6.06s
1000:	learn: 3.3533956	total: 8.68s	remaining: 4.33s
1200:	learn: 3.2439068	total: 10.5s	remaining: 2.61s
1400:	learn: 3.1545246	total: 12.3s	remaining: 867ms
1499:	learn: 3.1116048	total: 13.4s	remaining: 0us


In [ ]:
pred_test = test_fe.copy()
pred_test["predicted"] = test_pred

submission = sample_sub[["id"]].copy()

submission = submission.merge(
    pred_test[["id", "predicted"]],
    on="id",
    how="left"
)

submission.to_csv("submission.csv", index=False)
print("submission.csv saved!")

submission.csv saved!
